In [49]:
import numpy as np
import plotly.graph_objects as go
import sys
sys.path.append('../../assets/python/')
import tfb
from dice_roller import *

METADATA = {'Contributor': 'T. Dunn'}
SAVEFIGS = False

In [50]:
# Attack damage distribution with one or two attacks

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='value',
            range=[0,21],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='probability',
            tickformat='0.2f',
        ),
        legend=dict(
            xanchor='right', yanchor='top',
            x=1.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

# one attacks
"""for n in [4,6,8,10,12,20]:
    roll = Dice_Roller(f'1d{n}').value
    fig.add_trace(go.Scatter(
        x=np.array(roll.outcomes()), 
        y=np.array(roll.probabilities()),
        name=f'd{n}',
        mode='markers+lines',
        line_shape='hvh',
        fill='tozeroy',
    ))"""

x_list = list(range(22))
for deq in ['d4','d6','d8','d10','d12','d20']:
    roll = Dice_Roller(deq).value
    fig.add_trace(go.Scatter(
        x=np.array(x_list),
        y=np.array([roll.probability(x) for x in x_list]),
        #name=f'{deq} (std dev {roll.sd():.2f})',
        name=f'{deq}',
        mode='lines',
        line_shape='hvh',
        fill='tozeroy',
        hovertemplate=
            '<b>'+deq+'</b><br>'+
            'roll %{x:.0f}<br>'+
            'probability %{y:,.1%}' + 
            '<extra></extra>'
    ))

# show figure
fig.update_layout(width=600, height=450, barmode='group')
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-die-pd-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-die-pd-small')

In [51]:
# creates a table with mean, sigma, and sigma/mean values for common dice
import pandas as pd
import numpy as np

n = np.array([1, 4, 6, 8, 10, 12, 20])
m = 0.5*(n + 1)
s = np.sqrt((n*n - 1)/12)

df = pd.DataFrame({'die': [f'd{d}' for d in list(n)], 'mean': list(m), 'std dev': list(s)})
df['std dev/mean'] = df['std dev']/df['mean']

df = df.style.format({
    'mean': '{:,.1f}',
    'std dev': '{:,.2f}',
    'std dev/mean': '{:,.2f}',
}).hide().set_table_styles([
        {'selector': 'th', 'props': 'text-align: center;'},
        {'selector': 'td', 'props': 'text-align: center;'},
    ], overwrite=False)
df.to_html(
    './tab-die-stats.html',
    index=False, classes='wide', border=0)
df

die,mean,std dev,std dev/mean
d1,1.0,0.00,0.00
d4,2.5,1.12,0.45
d6,3.5,1.71,0.49
d8,4.5,2.29,0.51
d10,5.5,2.87,0.52
d12,6.5,3.45,0.53
d20,10.5,5.77,0.55


In [52]:
# average damage vs variance
# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='mean',
            range=[0, 40],
            tick0=0, dtick=10,
            minor=dict(tick0=0, dtick=5),
            showspikes=True,
            spikesnap='cursor',
            spikecolor='black', 
            spikethickness=1,
        ),
        yaxis=dict(
            title_text='standard deviation',
            range=[0, 10],
            tick0=0, dtick=2,
            minor=dict(tick0=0, dtick=1),
            showspikes=True,
            spikesnap='cursor',
            spikecolor='black', 
            spikethickness=1,
            #tickformat='0.1f',
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        ),
    )
)

for cv in [0.1,0.2,0.4,0.6]:
    angle = np.arctan(cv)
    r = [0,15,32,60]
    text = [''] + 2*[f'{cv:.2f}'] + ['']
    b = 10
    a = 40
    rp = (np.array(r)/40)*a*b/np.sqrt((b*np.cos(angle))**2 + (a*np.sin(angle))**2)
    xp = rp*np.cos(angle)
    yp = rp*np.sin(angle)
    fig.add_trace(go.Scatter(
        x=xp, 
        y=yp,
        name=f'CV {cv:.2f}',
        mode='lines+text',
        line=dict(dash='dashdot', color='black', width=1),
        showlegend=False,
        hoverinfo='skip',
    ))
    fig.add_annotation(
        x=xp[2], 
        y=yp[2],
        showarrow=False,
        text=f'{cv:.2f}',
        bgcolor='white',
    )

for d, s in zip([4,6,8,10,12],['triangle-up','square','diamond','pentagon','hexagon']):
    x = []
    y = []
    t = []
    ht = []
    for n in range(1, 20):
        deq = f'{n}d{d}'
        roll = Dice_Roller(deq).value
        x.append(roll.mean())
        y.append(roll.sd())
        t.append(deq)
        ht.append(f'{deq}<br>mean {roll.mean():.2f}<br>std dev {roll.sd():.2f}<br>CV {roll.sd()/roll.mean():.2f}')

    fig.add_trace(go.Scatter(
        #x=np.array(roll.outcomes()), 
        #y=np.array(roll.probabilities()),
        x=x,
        y=y,
        name=f'd{d}',
        mode='markers+lines',
        hoverinfo='text',
        hovertext=ht,
        marker=dict(size=10, symbol=s, line=dict(width=1, color='black')),
    ))

# show figure
fig.update_layout(width=600, height=500)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-cv-die-sigma-vs-mean-large', style='aspect-ratio:600/500;')
    tfb.save_fig_html(fig, format='small', name=f'./fig-cv-die-sigma-vs-mean-small', style='aspect-ratio:600/500;')

In [53]:
# Compares the probability distribution of 4d4 + 1 and 2d8 + 2

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='value',
            range=[0,25],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            range=[-0.002,0.18],
            title_text='probability',
            tickformat='0.0%',
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

def guassian(x, m, s):
    return (1/(s*np.sqrt(2*np.pi)))*np.exp(-0.5*((x - m)/s)**2)

# one attacks
x_list = list(range(30))
for deq in ['1d10 + 10','3d6 + 5']:
    roll = Dice_Roller(deq).value
    fig.add_trace(go.Scatter(
        x=np.array(x_list),
        y=np.array([roll.probability(x) for x in x_list]),
        name=f'{deq} (mean {roll.mean():.1f}, std dev {roll.sd():.1f})',
        mode='lines',
        line_shape='hvh',
        fill='tozeroy',
        hovertemplate=
            '<b>'+deq+'</b><br>'+
            'result %{x:.0f}<br>'+
            'probability %{y:,.1%}' + 
            '<extra></extra>'
    ))

    """fig.add_trace(go.Scatter(
        x=np.array(x_list),
        y=guassian(np.array(x_list), roll.mean(), roll.sd()),
        mode='lines',
    ))"""

# show figure
fig.update_layout(width=600, height=450, barmode='group')
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-damage-roll-example-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-damage-roll-example-small')

In [54]:
# plot monster damage average CV vs CR
import pandas as pd

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='challenge rating',
            range=[0,30],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            range=[-0.002,0.40],
            title_text='CV',
            tickformat='0.2f',
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

# load data
df = pd.read_csv('../../assets/data/monster-damage-variability.csv')
df = df.astype({
    'monster': 'str', 
    'id': 'str',
    'CR': 'float32', 
    'feature': 'str', 
    'mean': 'float32', 
    'variance': 'float32', 
    'standard deviation': 'float32', 
    'sigma/mean': 'float32', 
})
dfG = df[['CR','mean','variance','standard deviation','sigma/mean']].groupby(['CR']).mean().reset_index()
dfG.sort_values('CR', inplace=True)
dfG = dfG[dfG['CR'].between(1,30)]

fig.add_trace(go.Scatter(
    x=dfG['CR'], 
    y=dfG['sigma/mean'],
    name=f'monsters',
    mode='markers+lines',
    customdata=dfG['mean'],
    hovertemplate=
        'CR %{x:.0f}<br>'+
        'CV %{y:,.2f}<br>' + 
        'damage %{customdata:,.1f}' + 
        '<extra></extra>'
))

# show figure
fig.update_layout(width=600, height=450, barmode='group')
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-monster-damage-cv-vs-cr-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-monster-damage-cv-vs-cr-small')

In [55]:
# plot monster damage average CV vs CR
import pandas as pd

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='mean',
            range=[0,40],
            tick0=0, dtick=10,
            minor=dict(tick0=0, dtick=5),
        ),
        yaxis=dict(
            range=[0,10],
            title_text='standard deviation',
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='h',
            tracegroupgap=0,
            traceorder='grouped',
            bgcolor='rgba(0,0,0,0)',
        )
    )
)

for cv in [0.1,0.2,0.4,0.6]:
    angle = np.arctan(cv)
    r = [0,22,40,60]
    text = 1*[''] + 1*[f'{cv:.2f}'] + 2*['']
    b = 10
    a = 40
    rp = (np.array(r)/40)*a*b/np.sqrt((b*np.cos(angle))**2 + (a*np.sin(angle))**2)
    xp = rp*np.cos(angle)
    yp = rp*np.sin(angle)
    fig.add_trace(go.Scatter(
        x=xp, 
        y=yp,
        name=f'CV {cv:.2f}',
        mode='lines+text',
        line=dict(dash='dashdot', color='black', width=1),
        showlegend=False,
        hoverinfo='skip',
    ))
    for i in range(len(text)):
        if len(text[i]) > 0:
            fig.add_annotation(
                x=xp[i], 
                y=yp[i],
                showarrow=False,
                text=text[i],
                bgcolor='white',
            )

for d, s in zip([4,6,8,10,12],['triangle-up','square','diamond','pentagon','hexagon']):
    x = []
    y = []
    t = []
    ht = []
    for n in range(1, 20):
        deq = f'{n}d{d}'
        roll = Dice_Roller(deq).value
        x.append(roll.mean())
        y.append(roll.sd())
        t.append(deq)
        ht.append(f'{deq}<br>mean {roll.mean():.2f}<br>std dev {roll.sd():.2f}<br>CV {roll.sd()/roll.mean():.2f}')

    fig.add_trace(go.Scatter(
        x=x,
        y=y,
        name=f'd{d}',
        legendgroup='2',
        mode='markers+lines',
        hoverinfo='text',
        hovertext=ht,
        marker=dict(size=6, symbol=s, line=dict(width=0, color='black')),
    ))

# load data
df = pd.read_csv('../../assets/data/monster-damage-variability.csv')
df = df.astype({
    'monster': 'str', 
    'id': 'str',
    'CR': 'float32', 
    'feature': 'str', 
    'mean': 'float32', 
    'variance': 'float32', 
    'standard deviation': 'float32', 
    'sigma/mean': 'float32', 
})
dfG = df[['CR','mean','variance','standard deviation','sigma/mean']].groupby(['CR']).median().reset_index()
dfG.sort_values('CR', inplace=True)
dfG = dfG[dfG['CR'].between(1,30)]

fig.add_trace(go.Scatter(
    x=dfG['mean'], 
    y=dfG['standard deviation'],
    name=f'monsters',
    legendgroup='1',
    mode='markers',
    marker=dict(color='#636EFA', size=8, line=dict(width=1, color='black')),
    customdata=dfG['CR'],
    hovertemplate=
        'CR %{customdata:.0f}<br>'+
        'mean %{x:.1f}<br>'+
        'std dev %{y:,.1f}' + 
        '<extra></extra>'
))

# show figure
fig.update_layout(width=600, height=500, barmode='group')
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-monster-damage-sigma-vs-mean-large', style='aspect-ratio:600/500;', selector=dict(name='monsters'))
    tfb.save_fig_html(fig, format='small', name=f'./fig-monster-damage-sigma-vs-mean-small', style='aspect-ratio:600/500;', selector=dict(name='monsters'))